# KNN Imputation: Terry Stops Dataset

Now we're going to work on seeing how well KNN Imputation works on a dataset that is composed mostly of categorical data.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns